In [1]:
import numpy as np
from lib.func import fetch

In [ ]:
data = '''.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\\
..../.\\\..
.-.-/..|..
.|....-|.\\
..//.|....'''.split('\n')


In [2]:
data = fetch(16).split('\n')

### Functions

In [3]:
# Update coordinate with direction
def getNewCoord(beam):
    return list(np.add(beam['coord'], beam['dir']))


# Build new beam from directions and coordinate
def buildBeam(x, y, coord):
    return { 'dir': [x, y], 'coord': [coord[0], coord[1]]  }


# Check if new coordinate is within valid range of matrix
def isValidRange(coord, n):
    if coord[0] < 0 or coord[1] < 0:
        return False
    if coord[0] == n or coord[1] == n:
        return False
    return True


# Check if coordinate has been energized or 'lit' yet
def isLit(lit, coord):
    return lit.get(coord[0]).get(coord[1]) if lit.get(coord[0]) else None


# Split beam into two diverging vertical lines, return false to signify invalid old beam
def splitBeamVertical(beams, coord):
    beams.append(buildBeam(1, 0, coord))
    beams.append(buildBeam(-1, 0, coord))
    return False


# Split beam into two diverging horizontal lines, return false to signify invalid old beam
def splitBeamHorizontal(beams, coord):
    beams.append(buildBeam(0, 1, coord))
    beams.append(buildBeam(0, -1, coord))
    return False


# Convert direction for back slash with inverse
def convertDirBackSlash(beam):
    beam['dir'] = [beam['dir'][1], beam['dir'][0]]


# Convert direction for foreward slash with negative inverse
def convertDirFwdSlash(beam):
    beam['dir'] = [-beam['dir'][1], -beam['dir'][0]]


# Handle encountering slash
def handleSlash(beam, cell):
    if cell == '\\':
        convertDirBackSlash(beam)
    if cell == '/':
        convertDirFwdSlash(beam)


# Determine how beam moves based on new cell, return validity of current beam
def updateBeam(beams, beam, coord, cell, wasLit):
    # If it approaches from horizontal
    if cell == '|' and beam['dir'][1]:
        if wasLit:
            return False
        return splitBeamVertical(beams, coord)

    # If it approaches from vertical
    if cell == '-' and beam['dir'][0]:
        if wasLit:
            return False
        return splitBeamHorizontal(beams, coord)

    # Set new coordinate
    beam['coord'] = coord
    
    # Convert direction if slash
    handleSlash(beam, cell)
    return True
    

def moveBeam(data, beams, beam, lit):
    n = len(data)

    # Get next coordinate for beam
    [x, y] = getNewCoord(beam)

    # Check next coordinate for invalid range
    if not isValidRange([x, y], n):
        return False
    
    # If not lit, light next coord
    wasLit = isLit(lit, [x, y])
    if not wasLit:
        if not lit.get(x):
            lit.setdefault(x, {})
        lit.get(x).setdefault(y, True)

    cell = data[x][y]
    # Update beam with next coordinate
    return updateBeam(beams, beam, [x, y], cell, wasLit)


### Solution

In [4]:
# Get energized count by direction and starting coordinate
def getTotalEnergizedCount(data, d, c):
    # Create array to hold all beams
    beams = [{ 'dir': [d[0], d[1]], 'coord': [c[0], c[1]] }]
    lit = {}
    
    while len(beams):
        # Pop beam from list
        b = beams.pop()

        valid = True
        # While beam remains valid, move beam
        while valid:
            valid = moveBeam(data, beams, b, lit)
            
    # Sum up lengths of each lit row
    total = 0
    for key in lit:
        total += len(lit.get(key))
    return total

In [5]:
def solvePart2(data):
    n = len(data)
    most_lit = 0

    for i in range(n):
        vals = [
            most_lit,
            getTotalEnergizedCount(data, [1, 0], [-1, i]),
            getTotalEnergizedCount(data, [0, 1], [i, -1]),
            getTotalEnergizedCount(data, [-1, 0], [n, i]),
            getTotalEnergizedCount(data, [0, -1], [i, n])
        ]
        most_lit = max(vals)
        
    return most_lit



In [7]:
print(f'Part 1: {getTotalEnergizedCount(data, [0, 1], [0, -1])}')
print(f'Part 2: {solvePart2(data)}')

Part 1: 7608
Part 2: 8221
